# 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

tcga = pd.read_csv("merged_df.csv")
tcga_TAN = pd.read_csv("C:/Users/User/OneDrive/바탕 화면/0521 final code/ic50에서 파생/TANESPIMYCIN/TANESPIMYCIN_predictions.csv")
tcga_PEV = pd.read_csv("C:/Users/User/OneDrive/바탕 화면/0521 final code/ic50에서 파생/PEVONEDISTAT/PEVONEDISTAT_predictions.csv")
tcga_VIN = pd.read_csv("C:/Users/User/OneDrive/바탕 화면/0521 final code/ic50에서 파생/VINBLASTINE/VINBLASTINE_predictions.csv")
tcga_SORA = pd.read_csv("C:/Users/User/OneDrive/바탕 화면/0521 final code/ic50에서 파생/SORAFENIB/SORAFENIB_predictions.csv")
tcga_VINO = pd.read_csv("C:/Users/User/OneDrive/바탕 화면/학회/주헌 모델링/VINORELBINE/VINORELBINE_predictions.csv")

# **데이터 전처리**
 * 변수 선정
 * 변수명 변경

In [2]:
tcga_selected = tcga[['sample','AJCC_Stage_nature2012', 'PAM50_mRNA_nature2012','ER_Status_nature2012' , 'PR_Status_nature2012', 'HER2_Final_Status_nature2012']]

In [3]:
# 예: TCGA 샘플 ID가 'sample_id'라는 컬럼에 있을 경우
tcga_selected.set_index('sample', inplace=True)
tcga_selected.index = tcga_selected.index.str.replace("-", ".", regex=False)


In [4]:
tcga_selected

,AJCC_Stage_nature2012,PAM50_mRNA_nature2012,ER_Status_nature2012,PR_Status_nature2012,HER2_Final_Status_nature2012
sample,,,,,
TCGA.3C.AAAU.01,NaN,NaN,NaN,NaN,NaN
TCGA.3C.AALI.01,NaN,NaN,NaN,NaN,NaN
TCGA.3C.AALJ.01,NaN,NaN,NaN,NaN,NaN
TCGA.3C.AALK.01,NaN,NaN,NaN,NaN,NaN
TCGA.4H.AAAK.01,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
TCGA.WT.AB44.01,NaN,NaN,NaN,NaN,NaN
TCGA.XX.A899.01,NaN,NaN,NaN,NaN,NaN
TCGA.XX.A89A.01,NaN,NaN,NaN,NaN,NaN


In [5]:
# 확인할 대상 열 목록
cols = [
    'AJCC_Stage_nature2012',
    'PAM50_mRNA_nature2012',
    'ER_Status_nature2012',
    'PR_Status_nature2012',
    'HER2_Final_Status_nature2012'
]

# 각 열별 범주값 및 빈도 출력
for col in cols:
    print(f"--- {col} ---")
    print(tcga_selected[col].value_counts(dropna=False))
    print()


--- AJCC_Stage_nature2012 ---
AJCC_Stage_nature2012
NaN           437
Stage IIA     249
Stage IIB     156
Stage IIIA    102
Stage IA       64
Stage I        57
Stage II       40
Stage IIIC     32
Stage IIIB     23
Stage III      18
Stage IV       15
Stage IB       11
Stage X        10
Name: count, dtype: int64

--- PAM50_mRNA_nature2012 ---
PAM50_mRNA_nature2012
NaN              692
Luminal A        231
Luminal B        127
Basal-like        98
HER2-enriched     58
Normal-like        8
Name: count, dtype: int64

--- ER_Status_nature2012 ---
ER_Status_nature2012
Positive         599
NaN              434
Negative         179
Indeterminate      2
Name: count, dtype: int64

--- PR_Status_nature2012 ---
PR_Status_nature2012
Positive         521
NaN              435
Negative         254
Indeterminate      4
Name: count, dtype: int64

--- HER2_Final_Status_nature2012 ---
HER2_Final_Status_nature2012
Negative     650
NaN          440
Positive     114
Equivocal     10
Name: count, dtype: int64


In [6]:
# 1. 컬럼명 한글로 변경
tcga_selected = tcga_selected.rename(columns={
    'AJCC_Stage_nature2012': 'AJCC 병기',
    'PAM50_mRNA_nature2012': 'PAM50 서브타입',
    'ER_Status_nature2012': '에스트로겐 수용체 상태',
    'PR_Status_nature2012': '프로게스테론 수용체 상태',
    'HER2_Final_Status_nature2012': 'HER2 최종 상태'
})

# 2. 값 매핑 

# AJCC 병기: 'Stage ' 접두사만 제거
tcga_selected['AJCC 병기'] = (
    tcga_selected['AJCC 병기']
    .str.replace(r'^Stage ', '병기 ', regex=True)
)

# PAM50 서브타입
pam50_map = {
    'Luminal A': '루미널 A',
    'Luminal B': '루미널 B',
    'Basal-like': '기저형',
    'HER2-enriched': 'HER2 과발현형',
    'Normal-like': '정상유사형'
}
tcga_selected['PAM50 서브타입'] = (
    tcga_selected['PAM50 서브타입']
    .replace(pam50_map)
)

# 에스트로겐 수용체 상태
er_map = {
    'Positive': '수용체 있음',
    'Negative': '수용체 없음',
    'Indeterminate': '판정불가'
}
tcga_selected['에스트로겐 수용체 상태'] = (
    tcga_selected['에스트로겐 수용체 상태']
    .replace(er_map)
)

# 프로게스테론 수용체 상태
pr_map = {
    'Positive': '수용체 있음',
    'Negative': '수용체 없음',
    'Indeterminate': '판정불가'
}
tcga_selected['프로게스테론 수용체 상태'] = (
    tcga_selected['프로게스테론 수용체 상태']
    .replace(pr_map)
)

# HER2 최종 상태
her2_map = {
    'Positive': '양성',
    'Negative': '음성',
    'Equivocal': '미결정'
}
tcga_selected['HER2 최종 상태'] = (
    tcga_selected['HER2 최종 상태']
    .replace(her2_map)
)

# 변경 결과 확인
tcga_selected[['AJCC 병기',
               'PAM50 서브타입',
               '에스트로겐 수용체 상태',
               '프로게스테론 수용체 상태',
               'HER2 최종 상태']].head()


,AJCC 병기,PAM50 서브타입,에스트로겐 수용체 상태,프로게스테론 수용체 상태,HER2 최종 상태
sample,,,,,
TCGA.3C.AAAU.01,NaN,NaN,NaN,NaN,NaN
TCGA.3C.AALI.01,NaN,NaN,NaN,NaN,NaN
TCGA.3C.AALJ.01,NaN,NaN,NaN,NaN,NaN
TCGA.3C.AALK.01,NaN,NaN,NaN,NaN,NaN
TCGA.4H.AAAK.01,NaN,NaN,NaN,NaN,NaN


# **PEVONEDISTAT**

## 병합 및 민감/저항 그룹화

In [7]:
from scipy.stats import chi2_contingency
tcga_PEV.set_index('TCGA_ID' , inplace= True)
# 두 데이터프레임을 인덱스 기준으로 병합
PEV_merged =  tcga_selected.merge(tcga_PEV, left_index=True, right_index=True, how='inner')
PEV_merged.index

PEV_merged['pred'] = PEV_merged['pred'].map({0: '저항군', 1: '민감군'})
group_col = 'pred'  

## 카이제곱 검정 진행

In [10]:
# 결과 저장용
chi2_results = []
obj_cols = PEV_merged.select_dtypes(include='object').columns.drop(group_col, errors='ignore')
for col in obj_cols:
    contingency = pd.crosstab(PEV_merged[group_col], PEV_merged[col])
    
    if contingency.shape[1] > 1:  # 카이제곱은 2x2 이상일 때 유효
        chi2, p, dof, expected = chi2_contingency(contingency)
        chi2_results.append({
            'column': col,
            'p_value': p,
            'chi2_stat': chi2
        })

# 결과를 데이터프레임으로 보기 좋게
chi2_df = pd.DataFrame(chi2_results).sort_values('p_value')

print(chi2_df)

# 유의미한 변수만 필터링 (p < 0.05)
significant_cols = chi2_df[chi2_df['p_value'] < 0.05]['column'].tolist()
for col in significant_cols:
    print(f"\n  {col} (p = {chi2_df.loc[chi2_df['column'] == col, 'p_value'].values[0]:.4f})")
    
    ctab = pd.crosstab(PEV_merged[group_col], PEV_merged[col], margins=True, normalize='columns')
    display(ctab.round(3))  # 민감군 vs 저항군별 비율 보기


          column   p_value  chi2_stat
4     HER2 최종 상태  0.026082   7.293040
1     PAM50 서브타입  0.077744   6.823413
2   에스트로겐 수용체 상태  0.262888   2.672054
0        AJCC 병기  0.752891   5.043744
3  프로게스테론 수용체 상태  0.927122   0.008366

  HER2 최종 상태 (p = 0.0261)


HER2 최종 상태,미결정,양성,음성,All
pred,,,,
민감군,1.0,0.167,0.731,0.636
저항군,0.0,0.833,0.269,0.364


# **VINBLASTINE**

In [11]:
tcga_VIN.set_index('TCGA_ID' , inplace= True)
# 두 데이터프레임을 인덱스 기준으로 병합
VIN_merged =  tcga_selected.merge(tcga_VIN, left_index=True, right_index=True, how='inner')
VIN_merged.index

VIN_merged['pred'] = VIN_merged['pred'].map({0: '저항군', 1: '민감군'})
group_col = 'pred'  

obj_cols = VIN_merged.select_dtypes(include='object').columns.drop(group_col, errors='ignore')

In [13]:
# 결과 저장용
chi2_results = []

for col in obj_cols:
    contingency = pd.crosstab(VIN_merged[group_col], VIN_merged[col])
    
    if contingency.shape[1] > 1:  # 카이제곱은 2x2 이상일 때 유효
        chi2, p, dof, expected = chi2_contingency(contingency)
        chi2_results.append({
            'column': col,
            'p_value': p,
            'chi2_stat': chi2
        })

# 결과를 데이터프레임으로 보기 좋게
chi2_df = pd.DataFrame(chi2_results).sort_values('p_value')

print(chi2_df)

# 유의미한 변수만 필터링 (p < 0.05)
significant_cols = chi2_df[chi2_df['p_value'] < 0.05]['column'].tolist()
for col in significant_cols:
    print(f"\n  {col} (p = {chi2_df.loc[chi2_df['column'] == col, 'p_value'].values[0]:.4f})")
    
    ctab = pd.crosstab(VIN_merged[group_col], VIN_merged[col], margins=True, normalize='columns')
    display(ctab.round(3))  # 민감군 vs 저항군별 비율 보기


          column   p_value  chi2_stat
1     PAM50 서브타입  0.000992  18.484620
3  프로게스테론 수용체 상태  0.009476   9.318033
2   에스트로겐 수용체 상태  0.010328   9.145761
0        AJCC 병기  0.068174  18.624606
4     HER2 최종 상태  0.672224   0.794327

  PAM50 서브타입 (p = 0.0010)


PAM50 서브타입,HER2 과발현형,기저형,루미널 A,루미널 B,정상유사형,All
pred,,,,,,
민감군,0.429,0.793,0.382,0.867,0.25,0.596
저항군,0.571,0.207,0.618,0.133,0.75,0.404



  프로게스테론 수용체 상태 (p = 0.0095)


프로게스테론 수용체 상태,수용체 없음,수용체 있음,판정불가,All
pred,,,,
민감군,0.755,0.485,1.0,0.602
저항군,0.245,0.515,0.0,0.398



  에스트로겐 수용체 상태 (p = 0.0103)


에스트로겐 수용체 상태,수용체 없음,수용체 있음,판정불가,All
pred,,,,
민감군,0.8,0.512,1.0,0.602
저항군,0.2,0.488,0.0,0.398


# **VINORELBINE**

In [64]:
tcga_VINO.set_index('TCGA_ID' , inplace= True)

# 두 데이터프레임을 인덱스 기준으로 병합
VINO_merged =  tcga_selected.merge(tcga_VINO, left_index=True, right_index=True, how='inner')
VINO_merged.index

VINO_merged['pred'] = VINO_merged['pred'].map({0: '저항군', 1: '민감군'})
group_col = 'pred'  

# object형 열들만 선택 (group_col은 제외)
obj_cols = VINO_merged.select_dtypes(include='object').columns.drop(group_col, errors='ignore')

In [65]:

# 결과 저장용
chi2_results = []

for col in obj_cols:
    contingency = pd.crosstab(VINO_merged[group_col], VINO_merged[col])
    
    if contingency.shape[1] > 1: 
        chi2, p, dof, expected = chi2_contingency(contingency)
        chi2_results.append({
            'column': col,
            'p_value': p,
            'chi2_stat': chi2
        })

# 결과를 데이터프레임으로 보기 좋게
chi2_df = pd.DataFrame(chi2_results).sort_values('p_value')

print(chi2_df)

# 유의미한 변수만 필터링 (p < 0.05)
significant_cols = chi2_df[chi2_df['p_value'] < 0.05]['column'].tolist()
for col in significant_cols:
    print(f"\n▶ {col} (p = {chi2_df.loc[chi2_df['column'] == col, 'p_value'].values[0]:.4f})")
    
    ctab = pd.crosstab(VINO_merged[group_col], VINO_merged[col], margins=True, normalize='index')
    display(ctab.round(3))  # 민감군 vs 저항군별 비율 보기


          column   p_value  chi2_stat
0        AJCC 병기  0.027452  20.196078
4     HER2 최종 상태  0.367411   2.002550
1     PAM50 서브타입  0.769537   1.816090
3  프로게스테론 수용체 상태  0.868056   0.027599
2   에스트로겐 수용체 상태  1.000000   0.000000

▶ AJCC 병기 (p = 0.0275)


AJCC 병기,병기 I,병기 IA,병기 II,병기 IIA,병기 IIB,병기 III,병기 IIIA,병기 IIIB,병기 IIIC,병기 IV,병기 X
pred,,,,,,,,,,,
민감군,0.083,0.000,0.111,0.278,0.333,0.000,0.028,0.028,0.083,0.028,0.028
저항군,0.056,0.111,0.000,0.194,0.333,0.056,0.139,0.000,0.000,0.111,0.000
All,0.069,0.056,0.056,0.236,0.333,0.028,0.083,0.014,0.042,0.069,0.014


# **SORAFENIB**

In [66]:
tcga_SORA.set_index('TCGA_ID' , inplace= True)

# 두 데이터프레임을 인덱스 기준으로 병합
SORA_merged =  tcga_selected.merge(tcga_SORA, left_index=True, right_index=True, how='inner')
SORA_merged.index

SORA_merged['pred'] = SORA_merged['pred'].map({0: '저항군', 1: '민감군'})
group_col = 'pred'  

# object형 열들만 선택 (group_col은 제외)
obj_cols = SORA_merged.select_dtypes(include='object').columns.drop(group_col, errors='ignore')

In [73]:
# 결과 저장용
chi2_results = []

for col in obj_cols:
    contingency = pd.crosstab(SORA_merged[group_col], SORA_merged[col])
    
    if contingency.shape[1] > 1: 
        chi2, p, dof, expected = chi2_contingency(contingency)
        chi2_results.append({
            'column': col,
            'p_value': p,
            'chi2_stat': chi2
        })

# 결과를 데이터프레임으로 보기 좋게
chi2_df = pd.DataFrame(chi2_results).sort_values('p_value')

print(chi2_df)

# 유의미한 변수만 필터링 (p < 0.05)
significant_cols = chi2_df[chi2_df['p_value'] < 0.05]['column'].tolist()
for col in significant_cols:
    print(f"\n▶ {col} (p = {chi2_df.loc[chi2_df['column'] == col, 'p_value'].values[0]:.4f})")
    
    ctab = pd.crosstab(SORA_merged[group_col], SORA_merged[col], margins=True)
    display(ctab.round(3))  # 민감군 vs 저항군별 비율 보기


          column   p_value  chi2_stat
2   에스트로겐 수용체 상태  0.000024  17.836015
1     PAM50 서브타입  0.000037  23.159722
3  프로게스테론 수용체 상태  0.000842  11.146667
0        AJCC 병기  0.500710   7.337354
4     HER2 최종 상태  1.000000   0.000000

▶ 에스트로겐 수용체 상태 (p = 0.0000)


에스트로겐 수용체 상태,수용체 없음,수용체 있음,All
pred,,,
민감군,5,28,33
저항군,10,1,11
All,15,29,44



▶ PAM50 서브타입 (p = 0.0000)


PAM50 서브타입,HER2 과발현형,기저형,루미널 A,루미널 B,All
pred,,,,,
민감군,2,1,16,5,24
저항군,0,5,0,0,5
All,2,6,16,5,29



▶ 프로게스테론 수용체 상태 (p = 0.0008)


프로게스테론 수용체 상태,수용체 없음,수용체 있음,All
pred,,,
민감군,9,24,33
저항군,10,1,11
All,19,25,44


# **TANESPIMYCIN**

In [48]:
tcga_TAN.set_index('TCGA_ID' , inplace= True)
# 두 데이터프레임을 인덱스 기준으로 병합
TAN_merged =  tcga_selected.merge(tcga_VIN, left_index=True, right_index=True, how='inner')
TAN_merged.index

TAN_merged['pred'] = TAN_merged['pred'].map({0: '저항군', 1: '민감군'})

group_col = 'pred'  
# object형 열들만 선택 (group_col은 제외)
obj_cols = TAN_merged.select_dtypes(include='object').columns.drop(group_col, errors='ignore')

In [49]:
# 결과 저장용
chi2_results = []

for col in obj_cols:
    contingency = pd.crosstab(TAN_merged[group_col], TAN_merged[col])
    
    if contingency.shape[1] > 1: 
        chi2, p, dof, expected = chi2_contingency(contingency)
        chi2_results.append({
            'column': col,
            'p_value': p,
            'chi2_stat': chi2
        })

# 결과를 데이터프레임으로 보기 좋게
chi2_df = pd.DataFrame(chi2_results).sort_values('p_value')

print(chi2_df)

# 유의미한 변수만 필터링 (p < 0.05)
significant_cols = chi2_df[chi2_df['p_value'] < 0.05]['column'].tolist()
for col in significant_cols:
    print(f"\n▶ {col} (p = {chi2_df.loc[chi2_df['column'] == col, 'p_value'].values[0]:.4f})")
    
    ctab = pd.crosstab(TAN_merged[group_col], TAN_merged[col], margins=True, normalize='index')
    display(ctab.round(3))  # 민감군 vs 저항군별 비율 보기


                         column   p_value   chi2_stat
1         AJCC_Stage_nature2012  0.062077   14.852543
0                        sample  0.464848  179.000000
6                       TCGA_ID  0.464848  179.000000
3          ER_Status_nature2012  0.619116    0.247112
4          PR_Status_nature2012  0.687638    0.161654
5  HER2_Final_Status_nature2012  0.737513    0.608944
2         PAM50_mRNA_nature2012  0.837898    1.436095
